In [3]:
import sys, pathlib
sys.path.insert(0, str(pathlib.Path.cwd().parent.parent))  # add repo root
from tsum import tsum
import torch
import json

from ndtools import fun_binary_graph as fbg # ndtools available at github.com/jieunbyun/network-datasets
from ndtools.graphs import build_graph
from pathlib import Path
import networkx as nx   


In [4]:
DATASET = Path("data") 

nodes = json.loads((DATASET / "nodes.json").read_text(encoding="utf-8"))
edges = json.loads((DATASET / "edges.json").read_text(encoding="utf-8"))
probs_dict = json.loads((DATASET / "probs.json").read_text(encoding="utf-8"))

# build base graph
G_base: nx.Graph = build_graph(nodes, edges, probs_dict)

# all edges ON (example); add node/edge 0s as needed
states = {eid: 1 for eid in edges.keys()}

k_val, status, _ = fbg.eval_global_conn_k(states, G_base, target_k=2)
print("k =", k_val, "status =", status)

k = 2 status = s


In [5]:
s_fun = lambda comps_st: fbg.eval_global_conn_k(comps_st, G_base, target_k=2)
row_names = list(edges.keys()) + ['sys']
n_state = 2  # binary states: 0, 1

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
probs = [[probs_dict[n]['0']['p'], probs_dict[n]['1']['p']] for n in row_names[:-1]]
probs = torch.tensor(probs, dtype=torch.float32, device=device)

In [ ]:
result = tsum.run_rule_extraction(
    # Problem-specific callables / data
    sfun=s_fun,
    probs=probs,
    row_names=row_names,
    n_state=n_state,
    output_dir="tsum_res",
    surv_json_name="rules_surv.json",
    fail_json_name="rules_fail.json"
) 

Round: 1, Unk. prob.: 1.000e+00
No. of non-dominant rules: 0, Survival rules: 0, Failure rules: 0
Survival sample found from survival rules 👍
No. of existing rules removed:  0
New rule added. System state: s, flow: 2. Total samples: 1024.
New rule (No. of conditions: 11): {'e01': ('>=', 1), 'e02': ('>=', 1), 'e03': ('>=', 1), 'e04': ('>=', 1), 'e05': ('>=', 1), 'e06': ('>=', 1), 'e07': ('>=', 1), 'e08': ('>=', 1), 'e09': ('>=', 1), 'e10': ('>=', 1), 'e11': ('>=', 1), 'sys': ('>=', 1)}
Updated sys_vals: [2]
Failure sample found from failure rules 👍
No. of existing rules removed:  0
New rule added. System state: f, flow: 0. Total samples: 1024.
New rule (No. of conditions: 11): {'e01': ('<=', 0), 'e02': ('<=', 0), 'e03': ('<=', 0), 'e04': ('<=', 0), 'e05': ('<=', 0), 'e06': ('<=', 0), 'e07': ('<=', 0), 'e08': ('<=', 0), 'e09': ('<=', 0), 'e10': ('<=', 0), 'e11': ('<=', 0), 'sys': ('<=', 0)}
Unique system values: [2, 0]
Round: 2, Unk. prob.: 1.000e+00
No. of non-dominant rules: 2, Surviva